In [ ]:
fits_path = /home/dev/development/data-experiments/.data/illustris_data/342447.hdf5

In [ ]:
import numpy as np
import pandas as pd
import math
from spectral_cube import SpectralCube

def fits_to_dataframe(path):
    # Load the spectral cube
    cube = SpectralCube.read(path)

    df_list = []

    # Iterate over the spectral axis (velocity axis)
    for i in range(cube.shape[0]):
        # Slice one spectral frame at a time
        slab = cube[i, :, :]  # shape: (y, x)

        # Get world coordinates for this frame
        world = slab.wcs.pixel_to_world_values(
            *np.meshgrid(
                np.arange(cube.shape[2]),  # x (RA)
                np.arange(cube.shape[1]),  # y (Dec)
                indexing="xy",
            )
        )

        ra = world[0].flatten()
        dec = world[1].flatten()
        velo = cube.spectral_axis[i].value  # Single velocity value for this slice
        intensity = slab.filled_data[:].value.flatten()
        df_slice = pd.DataFrame(
            {"velocity": velo, "ra": ra, "dec": dec, "intensity": intensity}
        )

        df_list.append(df_slice)

    # Concatenate all slices into a single DataFrame
    df = pd.concat(df_list, ignore_index=True)
    df.dropna(inplace=True)

    
    del cube

    return df


: 

In [ ]:
df = fits_to_dataframe(fits_path)
df = df[df["intensity"]>0]

In [ ]:
df

,velocity,ra,dec,intensity
0,454133.415624,150.238422,2.336732,0.000111
1,453759.687091,150.238735,2.336732,0.000088
2,454047.614081,150.238590,2.336732,0.000060
3,453979.389644,150.238534,2.336732,0.000050
4,453872.734459,150.238615,2.336732,0.000069
...,...,...,...,...
4484829,-610719.758524,150.236196,2.339494,0.000103
4484830,-610646.475516,150.236076,2.339494,0.000084
4484831,-610752.687085,150.235948,2.339494,0.000062
4484832,-610716.359222,150.236090,2.339494,0.000037
